## Fluid-Strcuture Interaction

Fluid-Structure interaction is a multiphysics phenomenon consisting of coupled interaction of fluid and solid domains that are coupled at the interface, as shown below. The color schemes - blue for the fluid domain, red for the solid domain and black for the interface - are used henceforth.
<img src="./figures/FSI-basic.png" alt="drawing" width="600"/>

The fluid can be a simple single-phase creepage flow (Stokes flow) or it can be multiphase flow with turbulence and heat transfer. For example, we can consider the flow to be laminar, Newtonian, isothermal and incompressible, which can modelled using Navier-Stokes equations given as
\begin{align}
\rho^f \, \frac{D\mathbf{v}^f}{Dt} - \nabla \cdot \boldsymbol{\sigma}^f &= \mathbf{b}^f, \\
%
\nabla \cdot \mathbf{v}^f &= 0.
\end{align}

The solid can be a simple linear elastic solid with small deformation or it can be undergoing large deformations and fracture. For rigid solids, the governing equations are given as
\begin{align}
\mathbf{M} \, \ddot{\mathbf{d}} + \mathbf{C} \, \dot{\mathbf{d}} + \mathbf{K} \, \mathbf{d} &= \mathbf{F}, \\
%
\mathbf{d}(t=0) &= \mathbf{d}_0, \\
%
\dot{\mathbf{d}}(t=0) &= \dot{\mathbf{d}}_0
\end{align}

The fluid and solid domains are coupled at the interface by two conditions:

1.) Kinematic constrint:
\begin{equation}
\mathbf{v}^f = \dot{\mathbf{d}}
\end{equation}

2.) Traction equilibrium:
\begin{equation}
\boldsymbol{\sigma}^f \cdot \mathbf{n}^f + \boldsymbol{\sigma}^s \cdot \mathbf{n}^s = 0
\end{equation}
under the absence of surface tension. Here, $\mathbf{n}^f$ and $\mathbf{n}^s$ are the unit outward normals at the boundaries of fluid and solid domains, respectively.


## Solution schemes for FSI

There are various numerical methods for solving the combined equations of the FSI problem. The equations can be solved together, at once, or they can be solved in a segregated manner. Each method has its advantages and disadvantages over other methdos. Different schemes proposed in the literature FSI problem can be clubbed into two groups: (i) monolithic scheme and (ii) other (non-monolithic) schemes.


### Monolithic scheme
In a monolithic scheme, the governing equations of the FSI problem are solved together. That is, we have *one matrix* for the whole problem. The global matrix can be written as

\begin{align}
\begin{bmatrix}
\mathbf{K}^{ff}  &  \mathbf{K}^{fs} \\
\mathbf{K}^{sf}  &  \mathbf{K}^{ss} \\
\end{bmatrix}
%
\begin{Bmatrix}
\mathbf{U}^f  \\
\mathbf{U}^s
\end{Bmatrix}
=
-
\begin{Bmatrix}
\mathbf{R}^f  \\
\mathbf{R}^s
\end{Bmatrix}
\end{align}
where $\mathbf{K}^{ff}$, $\mathbf{R}^f$ and $\mathbf{U}^f$, respectively, are the matrix, residual and DOFs for the fluid problem, and $\mathbf{K}^{ss}$, $\mathbf{R}^s$ and $\mathbf{U}^s$ are the corresponding entries for the solid problem. $\mathbf{K}^{fs}$ and $\mathbf{K}^{sf}$ are the coupling matrices.

Typical matrix pattern of the $2 \times 2$ block matrix looks like
![](./figures/FSI-monolithic-scheme.png)



#### Advantages
* The coupling is ***strong***.
* No added-mass instabilities - possible to simulate even zero-density solids.
* Possible to achieve higher-order ($>=\mathcal{O}(\Delta ^2)$) temporaly accuracy.
* Solutions can be obtained with larger time step sizes.

#### Disadvantages
* Difficulty in linearising nonlinear equations and interface terms.
* Difficulty in developing scalable parallel preconditioners.
* Almost impossible to compute the coupling terms when using blackbox (third-party) solvers.


### Non-monolithic schemes
Another way of solving the equations of FSI problems are by splitting or segregaring them based on subdomains. That is, to solve the fluid and solid subdomains separately but by passing the required data between the solid and fluid subdomains. This can be illustrated as

![](./figures/FSI-staggered-scheme.png)

In the literature on computational FSI, numerous non-monolithic schemes have been proposed to solve the FSI problems in a segrerated manner, thereby enabling the coupling of (third-party) solvers developed for individual (fluid and solid) subdomains. Some of the names available in the literature are

* Partitioned schemes
* Split schemes
* Weakly-coupled schemes
* Loosely-coupled schemes
* Staggered schemes
* Explicitly-coupled schemes
* Iterative schemes

The term *staggered scheme* is historically used to denote an FSI scheme in which the solid and fluid problems are solved only once at each time steps. Other schemes which involve iterations are called as *partitioned schemes*.
Unfortunately, there is no established (or agreed) definiton for these terms. They are used *loosely*, making it difficult to categorise different schemes.

#### Advantages
* No need to compute the coupling matrices.
* Flexibility in coupling various third-party solvers for fluid and solid problems.
* Specialised parallel solvers and preconditioners for each problem.

#### Disadvantages
* Added-mass instabilities - stability depends on the relative densities of solid and fluid problems.
* Difficult to develop schemes with higher-order ($>=\mathcal{O}(\Delta ^2)$) temporal accuracy and wider stability zones.
* Reduced accuracy - (relatively) smaller time steps when compared to the monolithic scheme.


## Added-mass instabilities

* A symptom of loose/weak/explicit nature of the coupling.
* It’s not just the density ratio that affects the stability, geometry (thickness) also plays a vital role.
* The stability can be improved somewhat by using iterations. But, lower the density ratio ($ρ^s/ρ^f$) or thinner the geometry, the higher the added-mass and the higher the number of iterations.
* So many variations among partitioned schemes, each with different stability limits. The exact details of particular scheme matter a lot.


## Types of coupling

* First-order schemes require smaller time steps → expensive.
* The higher the accuracy the narrower the zone of stability.
* Non-intrusive, higher-order accurate staggered schemes with wider stability zones are still an open problem.

### Dirichlet-Neumann coupling
Dirichlet (velocity) BC on the fluid problem and Neumann (Force/traction) BC on the solid problem. Since these BCs are the standard BCs, this scheme is quite suitable for coupling different fluid and solid solvers as blackboxes. Partitioned schemes based on this coupling are known to suffer from severe added-mass instabilities.

### Dirichlet-Robin and Robin-Robin coupling
Dirichlet or Robin BC on the fluid problem and Robin BC on the solid problem. Since these BCs are NOT the standard BCs, these schemes are not quite suitable for coupling different fluid and solid solvers as blackboxes. These schemes are shown to have improve added-instabilities. However, since they fundamentally alter the dynamics of the subproblems, they have significant effect of the overall solution.

### Force-based coupling
Force-based coupling is the type of coupling used in the immersed boundary methods. Due to the way the interface condition is enforced, force-based coupling is the weakest type of coupling. However, force-based coupling can accommodate solids with zero masses, which is not possible in other types of coupling.

#### Questions
* Which sub-problem to solve first - solid or fluid?
* Which variable to predict - displacement, velocity, force?
* What is the condition for stability?
* Smallest mass/density ratio for stability.
* What is the order of accuracy?
* First-order schemes require smaller time steps → expensive.
* The higher the accuracy the narrower the zone of stability.
* Non-intrusive, higher-order accurate staggered schemes with wider stability zones are still an open problem.

## Original model problem

To understand different coupling schemes for FSI, we use a simple linear model problem.

The model problem is a spring-mass-damper model that is widely used in field of vibrations.

![Original](./figures/sdof-model-original.png)

The governing equation for the spring-mass-damper model without external force is given by
\begin{equation}
m \, \ddot{d} + c \, \dot{d} + k \, d = 0
\end{equation}
where $m$ is the mass, $c$ is the damping coefficient, $k$ is the stiffness of the spring, $d$ is displacement, $\dot{d}$ is velocity, and $\ddot{d}$ is acceleration.

The governing equation is supplemented with the initial conditions
\begin{equation}
d(t=0) = d_0, \\
\dot{d}(t=0) = \dot{d}_0
\end{equation}

For this model problem, we can obtain the analytical solution which we can use for comparison. We can also solve this equation numerically, and the solution thus obtained serves as the solution with a monolithic scheme. Further details will follow.

## Modified model problem

The spring-mass-damper model is modified, as shown below, by splitting the mass into two parts

![Original](./figures/sdof-model-modified.png)

The mass is split such that
\begin{equation}
m = m^s + m^f
\end{equation}

In terms of mass ratio ($m^r$), we have
\begin{align}
m^s &= m \, \frac{m^r}{1+m^r}, \\
%
m^f &= m \, \frac{1}{1+m^r}, \\
\end{align}

We introduce another parameter
\begin{equation}
\alpha = \frac{m^r}{1+m^r}
\end{equation}

The modified problem can be viewed as two systems coupled together:

System 1: mass $m^s$ and spring $k$. This mimics the solid subdomain in an FSI problem.

System 2: mass $m^f$ and damper $c$. This mimics the fluid subdomain in an FSI problem.

\begin{align}
m \, \ddot{d} + c \, \dot{d} + k \, d &= 0, \\
%
\left[ m^s+m^f \right] \, \ddot{d} + c \, \dot{d} + k \, d &= 0, \\
%
\underbrace{m^s \, \ddot{d} + k \, d}_{F^s} + \underbrace{m^f \, \ddot{d} + c \, \dot{d}}_{F^f} &= 0,
\end{align}

We can split the above equation as
\begin{align}
m^s \, \ddot{d} + k \, d &= F^s, \\
%
m^f \, \dot{v}^f + c \, v^f &= F^f, \\
%
F^s + F^f &= 0, \\
%
\dot{d} &= v^f,
\end{align}
in which
* the first equation is the governing equation for the solid subdomain,
* the second equation is the governing equation for the fluid subdomain,
* the third equation is the force equilibrium or force balance equation between solid and fluid subdomains,
* the fourth equation is the kinemetric constraint, and it ensures that the velocity of the solid and fluid interface is the same.

We can rewrite the governing equations in terms of $\alpha$, damping ratio $\xi$ and natural frequency ($\omega$) as
\begin{align}
\alpha \, \ddot{d} + \omega^2 \, d &= f^s, \\
%
(1-\alpha) \, \dot{v}^f + 2 \, \xi \, \omega \, v^f &= f^f, \\
%
f^s + f^f &= 0, \\
%
\dot{d} &= v^f,
\end{align}
where
\begin{align}
\omega = \sqrt{\frac{k}{m}}, \quad 
\xi = \frac{c}{2 m \omega}, \quad
f^s = \frac{F^s}{m}, \quad
f^f = \frac{F^f}{m}
\end{align}

Using the modified system and the corresponding governing equations, we can study different FSI coupling schemes. Although the system is quite simple, its analysis provides valuable insights into the performance of various FSI coupling schemes. However, it must be pointed that the analogy is limited to FSI problems where fluid interacts with rigid solids. It's extension to FSI with flexible is rather cumbersome.

In the next parts, we look at the spectral properties of different schemes.